### Pre-talk notes for Speaker!
During talk:
* Minimise file browser
* move to this folder cd .\Documents\GitHub\working-with-twitter-data\
* Zoom in
* Clear cells
* Share public link -> * Share public link -> https://github.com/UKDataServiceOpen/working-with-twitter-data/blob/main/TidyingDemo.ipynb

Talk time - 20 minutes

# Twarc Tidying and Analysis
This notebook will cover the exploration, tidying up and some basic analysis of the data collected by the [TwarcDemo in this repo](https://github.com/UKDataServiceOpen/working-with-twitter-data/blob/main/TwarcDemo.ipynb)

We will be using the 1000 Vegan Tweets from "Veganuary" 2019. If I didn't show a Twarc demo before you can find these in `data/demoData.csv`, and if you didn't follow along there is a provided version of this data.

So let's import some packages and read it in. 

In [1]:
import pandas as pd # Our data manipulation library
import numpy as np # Support for matrices, and other table-like shapes

In [2]:
# Tweak default plotting styles
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
plt.rcParams.update({'font.size': 22,
                    'figure.figsize':(24,8)})

In [12]:
# Read our data into a dataframe using pandas
data = pd.read_csv('3monthCoughUK.csv')

# The head function prints out the first 5 rows.
data.head()

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
0,1476704208074289156,1476701470447874050,1.476703e+18,NaN,NaN,181521084,1.226233e+08,NaN,NaN,2021-12-30T23:58:13.000Z,...,"[-1.315192, 53.1109178, -1.229692, 53.148616]",Feature,0f4224b306ddf55e,Sutton in Ashfield,0f4224b306ddf55e,city,2022-02-16T06:36:23+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.9.2,NaN
1,1476702886050648077,1476701470447874050,1.476701e+18,NaN,NaN,181521084,1.815211e+08,NaN,NaN,2021-12-30T23:52:58.000Z,...,"[-1.315192, 53.1109178, -1.229692, 53.148616]",Feature,0f4224b306ddf55e,Sutton in Ashfield,0f4224b306ddf55e,city,2022-02-16T06:36:23+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.9.2,NaN
2,1476701874116083712,1476686827331461128,1.476687e+18,NaN,NaN,1352804201101991941,6.538482e+07,NaN,NaN,2021-12-30T23:48:57.000Z,...,"[-1.2894037, 52.5436614, -1.2730678, 52.5567908]",Feature,223a7c85e7e3ff1b,Stoney Stanton,223a7c85e7e3ff1b,city,2022-02-16T06:36:23+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.9.2,NaN
3,1476697453919408135,1476686090111275009,1.476697e+18,NaN,NaN,1318834717295300609,3.578058e+09,NaN,NaN,2021-12-30T23:31:23.000Z,...,"[0.0748022, 51.4087608, 0.1889376, 51.478931]",Feature,569c1d39d5de5775,Bexley,569c1d39d5de5775,city,2022-02-16T06:36:23+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.9.2,NaN
4,1476694735628779526,1476397481869160455,1.476397e+18,NaN,NaN,390000120,3.547992e+09,NaN,NaN,2021-12-30T23:20:35.000Z,...,"[-2.659936, 51.399367, -2.510844, 51.516387]",Feature,7f15dd80ac78ef40,Bristol,7f15dd80ac78ef40,city,2022-02-16T06:36:23+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.9.2,NaN


We can test our tweets are real by taking the ID from the first column and replacing the ID in any tweet we can find, which I will demo!

So we've got our data read in successfully, let's print out some of the tweet text to make sure they have something to do with veganism.

In [4]:
for index in [1,2,3,4,5]:
    print(data['text'][index])
    print('\n')


I sneeze a lot in general sometimes for no apparent reason.


@darrenmark69 Is sneezing the difference? Don't ever remember that being listed as a symptom for coN vid (not that I believe anything they say about it anyway)


@petewall41 A week!!! Ffs. I had the shivers bad yesterday evening, cold one minute and hot the next. Bad fatigue, weakness, stiff muscles. It's been loads of fun 🤦‍♂️


@waywardscots Definitely over 40, and had all those side effects since mine, yesterday morning. Working from home, and had to lie down for a couple of hours this afternoon because I didn't even have the strength to lift my arms to type. Joints aching, full body shivers and a horrific headache.


@ItsMeNatashaP She reckons she's coming to the end of it as she's been coughing for a few days. I done a test earlier and was negative. Never say never though 😂




In [5]:
# I always recommend running info() for basic type information.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7097 entries, 0 to 7096
Data columns (total 74 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     7097 non-null   int64  
 1   conversation_id                        7097 non-null   int64  
 2   referenced_tweets.replied_to.id        3692 non-null   float64
 3   referenced_tweets.retweeted.id         0 non-null      float64
 4   referenced_tweets.quoted.id            369 non-null    float64
 5   author_id                              7097 non-null   int64  
 6   in_reply_to_user_id                    3794 non-null   float64
 7   retweeted_user_id                      0 non-null      float64
 8   quoted_user_id                         369 non-null    float64
 9   created_at                             7097 non-null   object 
 10  text                                   7097 non-null   object 
 11  lang

In [13]:
# and decribe() for statistical info.
# data.describe()
# Or to supress scientific notation
data[['author.public_metrics.followers_count','public_metrics.like_count','public_metrics.retweet_count']].describe().apply(lambda s: s.apply('{0:.0f}'.format))

,author.public_metrics.followers_count,public_metrics.like_count,public_metrics.retweet_count
count,7097,7097,7097
mean,2560,6,0
std,7492,110,6
min,0,0,0
25%,384,0,0
50%,951,1,0
75%,2381,3,0
max,236203,8653,298


At this point I am thinking we have too many columns to analyse really. It's worth at this point asking if there is anything we could remove now. Though if we are exploring this may be premature.

At this point it's probably time to narrow down what we are doing, there is too much interesting information in this dataset.

I am going to keep the following:
* id - The Tweet ID
* created_at - The time the tweet was created
* text - the text that makes up a tweet
* author.id - the author ID
* author.created_at - when the users account was created
* author.username - the Twitter users username
* author.location - a self-defined location
* author.public_metrics.followers_count - Number of followers a user has
* geo.full_name - the full name describing a tweets geolocation
* public_metrics.like_count - number of likes on this tweet
* public_metrics.retweet_count - number of retweets on this tweer.

In [11]:
data = data[['id','created_at', 'text','author.id','author.created_at', 'author.username','author.location','author.public_metrics.followers_count','geo.full_name','public_metrics.like_count','public_metrics.retweet_count']]
data.head()

,id,created_at,text,author.id,author.created_at,author.username,author.location,author.public_metrics.followers_count,geo.full_name,public_metrics.like_count,public_metrics.retweet_count
0,1476704208074289156,2021-12-30T23:58:13.000Z,"@planet_leesti Many things make me sneeze, som...",181521084,2010-08-22T12:08:39.000Z,Traceyspacey1,NaN,1016,"Sutton in Ashfield, England",0,0
1,1476702886050648077,2021-12-30T23:52:58.000Z,I sneeze a lot in general sometimes for no app...,181521084,2010-08-22T12:08:39.000Z,Traceyspacey1,NaN,1016,"Sutton in Ashfield, England",0,0
2,1476701874116083712,2021-12-30T23:48:57.000Z,@darrenmark69 Is sneezing the difference? Don'...,1352804201101991941,2021-01-23T02:24:13.000Z,darren_buckler,"Leicester, England",1420,"Stoney Stanton, England",0,0
3,1476697453919408135,2021-12-30T23:31:23.000Z,@petewall41 A week!!! Ffs. I had the shivers b...,1318834717295300609,2020-10-21T08:41:30.000Z,Webbo2306,NaN,421,"Bexley, London",1,0
4,1476694735628779526,2021-12-30T23:20:35.000Z,"@waywardscots Definitely over 40, and had all ...",390000120,2011-10-13T09:34:39.000Z,WrittenInRain,"Bristol, UK",753,"Bristol, England",2,0


# Have covid-19 symptoms grown?
Those of us still checking the news will know that in late November we had the Omicron strain develop

In [14]:
df = pd.read_csv('owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


# Symtpoms over time

# Have we missed a symptom?

# Sentiment of symptoms over time?

## How is Veganism percieved on Twitter?
In order to answer this question we need to introduce sentiment analysis. This is quite easy to do in Python as complicated as it sounds. As with many complicated things, somebody has written a package to make this easy for us.


In [ ]:
# Import NLTK, the Natural Language package
import nltk
# Download the popular vader lexicon of words and sentiments.
nltk.download([
    "vader_lexicon",
])

# import the sentiment analyser.
from nltk.sentiment import SentimentIntensityAnalyzer

# Create a new sentiment analyser.
sia = SentimentIntensityAnalyzer()

# And write a function we can pass to our pandas function
def get_sentiment(string):
    return sia.polarity_scores(string)['compound']

With this package our sentiment scores are returned on a scale of -1 for fully negative, to +1 for fully positive.
So our below sentence "I love cats" has a sentiment of 0.6, we high positive sentiment. Whereas "I hate cats is lower in sentiment.

In [ ]:
# Test our sentiment package
get_sentiment('I love cats')

In [ ]:
# Test our sentiment package
get_sentiment('I hate cats')

In [ ]:
# Test our sentiment package
get_sentiment('I am cats')

In [ ]:
# sentiment by word demo function
def sentiment_by_word(string):
    for word in string.split(' '):
        print(word + ' -- ' + str(get_sentiment(word)))

sentiment_by_word('I love cats')

In [ ]:
# So let's apply this to our entire dataframe
data['sentiment'] = data['text'].apply(get_sentiment)

# print a few rows
for index in [0,1,2,3,4]:
    print('sentiment ' + str(data['sentiment'][index]))
    print(data['text'][index])
    print('\n')

### What can we do with our sentiment scores
To start with, let's find our highest sentiment tweet.


In [ ]:
data.sort_values(by='sentiment', ascending=False).head()

In [ ]:
topSentimentIndex = 156
print(data['id'][topSentimentIndex])
print(data['text'][topSentimentIndex])
print(data['sentiment'][topSentimentIndex])

In [ ]:
sentiment_by_word(data['text'][topSentimentIndex])

In [ ]:
# And how about the lowest sentiment?
data.sort_values(by='sentiment', ascending=True).head()

In [ ]:
bottomSentimentIndex = 104
print(data['id'][bottomSentimentIndex])
print(data['text'][bottomSentimentIndex])
print(data['sentiment'][bottomSentimentIndex])

In [ ]:
sentiment_by_word(data['text'][bottomSentimentIndex])

In [ ]:
# Let's also grab some neutral tweets.
data[data['sentiment'] == 0]

In [ ]:
neutralIndex = 3
print(data['id'][neutralIndex])
print(data['text'][neutralIndex])
print(data['sentiment'][neutralIndex])

In [ ]:
sentiment_by_word(data['text'][neutralIndex])

In [ ]:
# Often the users at each end of this spectrum are quite different, lets see what our tweeters look like in general
data.sentiment.hist()

At a glance there are three different kinds of tweets here:
1. Negative tweets, these are likely complaints from vegans or complaints about vegans.
2. Neutral tweets, Most of the users appear to be neutral, this is usually a symptom that our sentiment analyser wasn't trained on the language it's predicting on, so is seeing words it's never classified before and tags them as neutral.
3. Positive tweets, these appear in abundance in comparison to negativity. Could this be a sign of positivity, marketing, bias?

On a hunch this grouping is quite naive. I could imagine a big difference in group 1:
* People complaining about vegans in a hateful way
* Vegans complaining about non-vegans in a hateful way
* Vegans complaining about vegan difficulties.

group 3 likely contains:
* Inflated self-promotion from vegan business owners
* Inflated promotion and feedback from large brands launching vegan products such as Greggs and the Vegan sausage roll.

These are all much larger project ideas, we don't even have a level of condience that any of these users are vegan.

### Most liked content
We have access to likes and retweets, let's check out what the most liked content is.

In [ ]:
# A fairly familiar graph of likes being geometrically hard to gain, with outliers from "viral" tweets
data['public_metrics.like_count'].hist(bins=100)

In [ ]:
# And we see similar with retweets, even harder to come by as an echo of messaging rather than approval.
data['public_metrics.retweet_count'].hist(bins=20)

In [ ]:
# And how about the most liked tweet?
data.sort_values(by='public_metrics.like_count', ascending=False).head()

In [ ]:
mostLikedIndex = 1012
print(data['id'][mostLikedIndex])
print(data['text'][mostLikedIndex])
print(data['sentiment'][mostLikedIndex])

In [ ]:
sentiment_by_word(data['text'][mostLikedIndex])

In [ ]:
# And how about the most retweeted?
data.sort_values(by='public_metrics.retweet_count', ascending=False).head()

In [ ]:
mostRetweetedIndex = 1012
print(data['id'][mostRetweetedIndex])
print(data['text'][mostRetweetedIndex])
print(data['sentiment'][mostRetweetedIndex])

The most liked and most retweeted came from Boy George, there is a correlation there that we will look at later.

## Does the perception of Veganism change over time?
This is a toy example of what we saw in the presentation. How do we percieve a rolling sentiment over time.

Next let's sort this dataframe by date. Looking now we seem to have only a handful of minutes between our Tweets.

In [ ]:
# sort by date
# convert created at into a datetime object
data['created_at'] =pd.to_datetime(data.created_at)
# Sort our dataframe b dates
data = data.sort_values(by='created_at',ascending=True)
# reset index
data = data.reset_index()
del data['index']

# print the head
data.head()

In [ ]:
# To start with let's plot sentiment over index. Although this is linear, rather than time based it can be useful.
data['sentiment'].plot()

In [ ]:
# As each tweet is relatively unconnected we can get quite erratic plots, some smoothin can help.
data['sentiment'].rolling(30).mean().plot()

In [ ]:
# positive tweets
len(data[data['sentiment'] > 0])

In [ ]:
# neutral tweets
len(data[data['sentiment'] == 0])

In [ ]:
# negative tweets
len(data[data['sentiment'] < 0])

Most of our tweets are vegan, some or neutral, and few are negative.

Not much to see here, in the full veganuary dataset we see a kickoff of positively, a trend downward and then a celebration at the end of the month.

In [ ]:
# Our neutral tweets, generally introduce some high-sentiment content that our model doesn't understand yet, so maybe we should remove them.
data[data.sentiment != 0]['sentiment'].rolling(30).mean().plot()

In [ ]:
# plot with realistic time axis
data.plot(kind='scatter',x='created_at', y='sentiment')

As we are only looking at such a small sample it's hard to draw any conclusions from this data. Depending on what we follow this can be a very clear line that somewhat represents the sentiment of a topic over time. Diving into our neutral tweets to better classify could be a good next step.

### Does sentiment correlate with success?
Now we have quantified sentiment, do high sentiment messages get engagement?
Let's plot our:
* follower counts
* likes count
* retweet count
* sentiment 

And see what we find.

In [ ]:
corr = data[['author.public_metrics.followers_count','public_metrics.like_count','public_metrics.retweet_count','sentiment']].corr()
corr.style.background_gradient(cmap ='coolwarm')

My hunch is that as so many results have neutral sentiment this is probably shifting our correlations quite heavily, let's remove them for now.

In [ ]:
corr = data[['author.public_metrics.followers_count','public_metrics.like_count','public_metrics.retweet_count','sentiment']][data['sentiment'] != 0].corr()
corr.style.background_gradient(cmap ='coolwarm')

It seems that the more followers a user has, the more likely their content is to be liked and retweeted. Follower count does not correlate with the sentiment of tweets though.

Likes and retweets have a strong correlation, content that is likely to be retweeted is also likely to be liked.

In this case sentiment doesn't seem to correlate with any of these features though.

By this point, we have noticed that our neutral sentiment tweets are a bit of a missed oppurtunity. We understand sentiment generally, but do not understand the terms within our topic area. The word "carnivore" might be usual, but in vegan circles it can be used in disgust or even as an insult, our sentiment analyzer doesn't understand this.

A good next step would be trying to figure out what these words are, but I will leave this here for now.

## Future Work
* Getting a full word count from tweets
* Dealing with stop words, punctuation and hashtags
* Removing duplicate words through case sensitivity, fuzzy matching and stemming
* Making word clouds with [wordclouds.co.uk](https://www.wordclouds.co.uk/)
* Classifying types of tweet into marketing, self-promotion and true oppinion.

## Useful Links
* Word cloud builder - [wordclouds.co.uk](https://www.wordclouds.co.uk/)
* An intro to basic NLP and word clouds with WhatsApp data - [What can I do with WhatsApp?](https://towardsdatascience.com/what-can-i-do-with-whatsapp-661fc3cdd5c5)
* Use machine learning to understand and leverage text. - [Solving 90% of NLP](https://www.kdnuggets.com/2019/01/solve-90-nlp-problems-step-by-step-guide.html)